In [5]:
from glob import glob
import os
import shutil
import nibabel as nib
from scipy import ndimage
from seg_brain_mri import main as get_mask

In [6]:
data_dir = os.path.join('dataset3/test/')
mris = sorted(glob(os.path.join(data_dir, "*_MRI.nii.gz")))
for i in range(len(mris)):    
    outname = os.path.join(data_dir, mris[i].split('/')[-1].split('.')[0])
    get_mask("final/seg/brain_resample_irisfeminad.pth", mris[i], outname, False, False, False, True)

In [ ]:
def get_largest_component(seg):
    mask = seg
    data = mask.get_fdata()
    label_im, nb_labels = ndimage.label(data)
    sizes = ndimage.sum(data, label_im, range(nb_labels + 1))
    data = sizes == max(sizes)
    binary_img = data[label_im]
    new_mask = nib.Nifti1Image(binary_img, mask.affine, mask.header)
    return new_mask

In [ ]:
masks = sorted(glob(os.path.join(data_dir, 'label*')))
for i in range(len(mris)):
    mask = nib.load(masks[i])
    mask_largestcomponent = get_largest_component(mask)
    outname = masks[i].split('\\')[-1].split('.')[0] + '_largest.nii.gz'
    nib.save(mask_largestcomponent, os.path.join(data_dir, outname))

In [ ]:
masks = sorted(glob(os.path.join(data_dir, "*_largest.nii.gz")))
for i in range(len(mris)):
    mri = nib.load(mris[i])
    mask = nib.load(masks[i])
    mask = get_largest_component(mask)
    
    mri_data = mri.get_fdata().copy()
    mask_data = mask.get_fdata().copy()
    mriskullstripped_data = mri_data * mask_data
    
    header = mri.header.copy()
    affine = mri.affine.copy()
    mriskullstripped = nib.Nifti1Image(mriskullstripped_data, affine, header)
    outname = mris[i].split('\\')[-1].split('.')[0] + '_skullstripped.nii.gz'
    nib.save(mriskullstripped, os.path.join(data_dir, outname))
    